In [ ]:
import os, sys
import numpy as np
import pandas as pd

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import corner

# Sampling from the BNN posterior
import torch
import torchvision.models
from h0rton.configs import TrainValConfig, TestConfig
import h0rton.losses
from h0rton.h0_inference import *
import h0rton.tdlmc_data
import h0rton.tdlmc_utils
import h0rton.train_utils as train_utils
from h0rton.trainval_data import XYData, XYCosmoData
from torch.utils.data import DataLoader

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Visualizing the BNN posterior

__Author:__ Ji Won Park (@jiwoncpark)

__Created:__ 11/01/2019

__Last run:__ 1/07/2019

__Goals:__
We visualize the BNN posterior overlaid against the BNN prior.

__Before_running:__
1. Train the BNN, e.g.
```bash
python -m h0rton.train h0rton/example_user_config.py
```

In [ ]:
cfg = TrainValConfig.from_file(os.path.join('..', 'h0rton', 'train_val_config_file.py'))

In [ ]:
device = torch.device('cuda')
if device.type == 'cuda':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

loss_fn = getattr(h0rton.losses, cfg.model.likelihood_class)(Y_dim=cfg.data.Y_dim, device=device)
net = getattr(torchvision.models, cfg.model.architecture)(pretrained=False)
n_filters = net.fc.in_features
net.fc = torch.nn.Linear(in_features=n_filters, out_features=loss_fn.out_dim) # replace final layer
net.to(device)
# Load trained weights from saved state
net, epoch = train_utils.load_state_dict_test("/home/jwp/stage/sl/h0rton/overnight_models/resnet34_epoch=864_01-15-2020_04:36.mdl", net, cfg.optim.n_epochs, device)

In [ ]:
val_data = XYData(cfg.data.val_dir, data_cfg=cfg.data)
n_val = 400
val_loader = DataLoader(val_data, batch_size=n_val, shuffle=False, drop_last=True)

with torch.no_grad():
    net.eval()
    for _, (X_, Y_) in enumerate(val_loader):
        X = X_.to(device)
        Y = Y_.to(device)
        pred = net(X)
        break

In [ ]:
cfg.data.Y_cols_to_log_parameterize_idx

In [ ]:
bnn_post = DoubleGaussianBNNPosterior(val_data.Y_dim, device, cfg.data.Y_cols_to_whiten_idx, cfg.data.train_Y_mean, cfg.data.train_Y_std, cfg.data.Y_cols_to_log_parameterize_idx, )
bnn_post.set_sliced_pred(pred)

In [ ]:
#mu_orig = bnn_post.transform_back(bnn_post.mu).cpu().numpy()
#mu_orig2 = bnn_post.transform_back(bnn_post.mu2).cpu().numpy()

In [ ]:
import baobab.sim_utils
n_samples = 3000 # number of bnn samples per lens
bnn_samples = bnn_post.sample(n_samples, sample_seed=0)

In [ ]:
truth = bnn_post.transform_back(Y).cpu().numpy()

In [ ]:
lens_i = 1
bnn_sample_df = pd.DataFrame(bnn_samples[lens_i, :, :], columns=cfg.data.Y_cols)
truth_lens_i = truth[lens_i, :]
print(truth_lens_i)

In [ ]:
mu_orig = bnn_post.transform_back(bnn_post.mu).cpu().numpy()
mu_orig2 = bnn_post.transform_back(bnn_post.mu2).cpu().numpy()

In [ ]:
mu_orig[lens_i, :]

In [ ]:
param_idx = 1
plt.hist(bnn_samples[lens_i, :, param_idx], bins=30)
plt.axvline(mu_orig[lens_i, param_idx], color='r')
plt.axvline(mu_orig2[lens_i, param_idx], color='b')

In [ ]:
bnn_samples[lens_i, :, 0]

In [ ]:
prior_Y = torch.Tensor(val_data.Y_df[cfg.data.Y_cols].values)
prior_Y = bnn_post.transform_back(prior_Y).cpu().numpy()
print(prior_Y.shape)

In [ ]:
import matplotlib
matplotlib.rcParams['text.usetex'] = True
t = np.linspace(0.0, 1.0, 100)
s = np.cos(4 * np.pi * t) + 2

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.plot(t, s)

ax.set_xlabel(r"$\theta_E ('')$")

In [ ]:
bnn_post_fig = corner.corner(bnn_sample_df[cfg.data.Y_cols],
                             color='tab:red',
                             smooth=1.0,
                             alpha=0.5,
                             labels=cfg.data.Y_cols_latex_names,
                             no_fill_contours=True,
                             plot_datapoints=False,
                             plot_contours=True,
                             show_titles=True,
                             levels=[0.68, 0.95],
                             contour_kwargs=dict(linestyles='solid'),
                             quiet=True,
                             hist_kwargs=dict(density=True,))

bnn_prior_fig = corner.corner(prior_Y,
                              color='tab:orange',
                              smooth=1.0,
                              alpha=0.5,
                              no_fill_contours=True,
                              plot_datapoints=False,
                              plot_contours=True,
                              truths=truth_lens_i,
                              fig=bnn_post_fig,
                              truths_color='tab:green',
                              levels=[0.68, 0.95],
                              contour_kwargs=dict(linestyles='solid'),
                              quiet=True,
                              range=[0.99]*len(cfg.data.Y_cols),
                              hist_kwargs=dict(density=True, ))

In [ ]:
plus_minus = np.array(cfg.data.Y_cols_range)
lower = truth_lens_i - plus_minus
upper = truth_lens_i + plus_minus
display_range = list(zip(lower, upper))

In [ ]:
_ = corner.corner(bnn_sample_df[cfg.data.Y_cols],
                             color='tab:red',
                             smooth=1.0,
                             alpha=0.5,
                             labels=cfg.data.Y_cols_latex_names,
                             truths=truth_lens_i,
                              truths_color='tab:green',
                             no_fill_contours=True,
                             plot_datapoints=False,
                             plot_contours=True,
                             show_titles=True,
                             levels=[0.68, 0.95],
                              range=display_range,
                             contour_kwargs=dict(linestyles='solid'),
                             quiet=True,
                             hist_kwargs=dict(density=True,))